In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

## Data Processing

In [2]:
df = pd.read_csv('./trips_BGMM.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 262800 entries, 0 to 262799
Data columns (total 27 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Cluster              262800 non-null  int64  
 1   Year                 262800 non-null  int64  
 2   Month                262800 non-null  int64  
 3   Day                  262800 non-null  int64  
 4   Time                 262800 non-null  object 
 5   Weekend              262800 non-null  int64  
 6   Holiday              262800 non-null  int64  
 7   Checkin per Hour     262800 non-null  float64
 8   Checkout per Hour    262800 non-null  float64
 9   Temp (°C)            262800 non-null  float64
 10  Dew Point Temp (°C)  262800 non-null  float64
 11  Rel Hum (%)          262800 non-null  float64
 12  Wind Spd (km/h)      262800 non-null  float64
 13  Visibility (km)      262800 non-null  float64
 14  Stn Press (kPa)      262800 non-null  float64
 15  Blowing Snow     

In [4]:
# get df per cluster
res = pd.DataFrame(columns=['Cluster'])
dfs = {}
for c in range(df['Cluster'].max()+1):
    # cluster c
    res.loc[c, 'Cluster'] = c
    df_cur = df[df['Cluster'] == c].drop(columns=['Cluster', 'Year', 'Month', 'Day'])
    # get dummies for time
    dummies = pd.get_dummies(df_cur['Time'], prefix='Time')
    df_cur.drop(columns=['Time'], inplace=True)
    df_cur = pd.concat([df_cur, dummies], axis=1)
    dfs[c] = df_cur

## Ridge Regression 

In [5]:
from sklearn.linear_model import Ridge

In [6]:
# set pipeline
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('ridge', Ridge())])

In [7]:
# set grid search
params = {'ridge__alpha': [0, 1, 5, 10, 50, 100, 500, 1000]}
ridge_search = GridSearchCV(pipeline, params, cv=3, verbose=2, n_jobs=-1)

In [8]:
res['Ridge Regression'] = 0
for c in dfs:
    print('For cluster {}:'.format(c))
    # train test split
    df_train, df_val = train_test_split(dfs[c], test_size=0.2, random_state=1207)
    # get X Y
    X_train = df_train.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_train = df_train['Checkin per Hour'] - df_train['Checkout per Hour']
    X_val = df_val.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_val = df_val['Checkin per Hour'] - df_val['Checkout per Hour']
    # shuffle
    X_train, Y_train = shuffle(X_train, Y_train, random_state=1207)
    # train model
    ridge_search.fit(X_train, Y_train)
    print('Best params: {}'.format(ridge_search.best_params_))
    # predict
    Y_pred = ridge_search.predict(X_val)
    mse = mean_squared_error(Y_val, Y_pred)
    print('MSE: {}'.format(mse))
    r2 = ridge_search.score(X_val, Y_val)
    print('R^2: {}'.format(r2))
    res.loc[c, 'Ridge Regression'] = r2
    print()

For cluster 0:
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:    9.0s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    9.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'ridge__alpha': 100}
MSE: 58.29027628943926
R^2: 0.3477972435907811

For cluster 1:
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'ridge__alpha': 100}
MSE: 60.98337538366479
R^2: 0.20373643564951682

For cluster 2:
Fitting 3 folds for each of 8 candidates, totalling 24 fits


...

[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'ridge__alpha': 100}
MSE: 30.8249112186073
R^2: 0.3274020524290112

For cluster 28:
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'ridge__alpha': 100}
MSE: 21.98757596961035
R^2: 0.16462982441565222

For cluster 29:
Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best params: {'ridge__alpha': 1000}
MSE: 2.6873664773208845
R^2: 0.005124585942789084



[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    0.4s finished


In [9]:
# save csv
res.to_csv('./pred_res.csv', index=False)

## LASSO Regression

In [10]:
from sklearn.linear_model import Lasso

In [11]:
# set pipeline
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('lasso', Lasso())])

In [12]:
# set grid search
params = {'lasso__alpha': [1, 5, 10, 50, 100, 500, 1000]}
lasso_search = GridSearchCV(pipeline, params, cv=3, verbose=2, n_jobs=-1)

In [13]:
res['LASSO Regression'] = 0
for c in dfs:
    print('For cluster {}:'.format(c))
    # train test split
    df_train, df_val = train_test_split(dfs[c], test_size=0.2, random_state=1207)
    # get X Y
    X_train = df_train.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_train = df_train['Checkin per Hour'] - df_train['Checkout per Hour']
    X_val = df_val.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_val = df_val['Checkin per Hour'] - df_val['Checkout per Hour']
    # shuffle
    X_train, Y_train = shuffle(X_train, Y_train, random_state=1207)
    # train model
    lasso_search.fit(X_train, Y_train)
    print('Best params: {}'.format(lasso_search.best_params_))
    # predict
    Y_pred = lasso_search.predict(X_val)
    mse = mean_squared_error(Y_val, Y_pred)
    print('MSE: {}'.format(mse))
    r2 = lasso_search.score(X_val, Y_val)
    print('R^2: {}'.format(r2))
    res.loc[c, 'LASSO Regression'] = r2
    print()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


For cluster 0:
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'lasso__alpha': 1}
MSE: 65.1725775137879
R^2: 0.27079201879701986

For cluster 1:
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'lasso__alpha': 1}
MSE: 37.844848313530555
R^2: 0.17422739286104905

For cluster 28:
Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'lasso__alpha': 1}
MSE: 26.20611916094877
R^2: 0.00435544167654478

For cluster 29:
Fitting 3 folds for each of 7 candidates, totalling 21 fits
Best params: {'lasso__alpha': 1}
MSE: 2.703674925981527
R^2: -0.0009128766626320672



[Parallel(n_jobs=-1)]: Done  17 out of  21 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:    0.3s finished


In [14]:
# save csv
res.to_csv('./pred_res.csv', index=False)

## AdaBoost Regression

In [15]:
from sklearn.ensemble import AdaBoostRegressor

In [16]:
# set pipeline
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('ada', AdaBoostRegressor())])

In [17]:
# set grid search
params = {'ada__n_estimators': [10, 50, 100],
          'ada__learning_rate' : [0.01, 0.05, 0.1, 0.5],
          'ada__loss' : ['linear', 'square', 'exponential']}
ada_search = GridSearchCV(pipeline, params, cv=3, verbose=2, n_jobs=-1)

In [18]:
res['AdaBoost Regression'] = 0
for c in dfs:
    print('For cluster {}:'.format(c))
    # train test split
    df_train, df_val = train_test_split(dfs[c], test_size=0.2, random_state=1207)
    # get X Y
    X_train = df_train.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_train = df_train['Checkin per Hour'] - df_train['Checkout per Hour']
    X_val = df_val.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_val = df_val['Checkin per Hour'] - df_val['Checkout per Hour']
    # shuffle
    X_train, Y_train = shuffle(X_train, Y_train, random_state=1207)
    # train model
    ada_search.fit(X_train, Y_train)
    print('Best params: {}'.format(ada_search.best_params_))
    # predict
    Y_pred = ada_search.predict(X_val)
    mse = mean_squared_error(Y_val, Y_pred)
    print('MSE: {}'.format(mse))
    r2 = ada_search.score(X_val, Y_val)
    print('R^2: {}'.format(r2))
    res.loc[c, 'AdaBoost Regression'] = r2
    print()

For cluster 0:
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   30.9s finished


Best params: {'ada__learning_rate': 0.5, 'ada__loss': 'linear', 'ada__n_estimators': 100}
MSE: 47.796395972847264
R^2: 0.4652119841544613

For cluster 1:
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   28.4s finished


Best params: {'ada__learning_rate': 0.01, 'ada__loss': 'exponential', 'ada__n_estimators': 50}
MSE: 49.89889009189856
R^2: 0.34846722025899746

For cluster 2:
Fitting 3 folds for each of 36 candidates, totalling 108 fits


...

Best params: {'ada__learning_rate': 0.05, 'ada__loss': 'exponential', 'ada__n_estimators': 100}
MSE: 29.844730111952707
R^2: 0.34878955281492563

For cluster 28:
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   31.5s finished


Best params: {'ada__learning_rate': 0.05, 'ada__loss': 'linear', 'ada__n_estimators': 50}
MSE: 21.53750114568413
R^2: 0.18172943945320774

For cluster 29:
Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 108 out of 108 | elapsed:   28.8s finished


Best params: {'ada__learning_rate': 0.05, 'ada__loss': 'linear', 'ada__n_estimators': 10}
MSE: 2.698749594775771
R^2: 0.0009105035737622114



In [19]:
# save csv
res.to_csv('./pred_res.csv', index=False)

## KNN

In [20]:
from sklearn.neighbors import KNeighborsRegressor

In [21]:
# set pipeline
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('knn', KNeighborsRegressor())])

In [22]:
# set grid search
params = {'knn__n_neighbors': [3, 5, 11, 19, 23, 29],
          'knn__weights': ['uniform', 'distance'],
          'knn__metric': ['euclidean', 'manhattan']}
knn_search = GridSearchCV(pipeline, params, cv=3, verbose=2, n_jobs=-1)

In [23]:
res['KNN'] = 0
for c in dfs:
    print('For cluster {}:'.format(c))
    # train test split
    df_train, df_val = train_test_split(dfs[c], test_size=0.2, random_state=1207)
    # get X Y
    X_train = df_train.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_train = df_train['Checkin per Hour'] - df_train['Checkout per Hour']
    X_val = df_val.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_val = df_val['Checkin per Hour'] - df_val['Checkout per Hour']
    # shuffle
    X_train, Y_train = shuffle(X_train, Y_train, random_state=1207)
    # train model
    knn_search.fit(X_train, Y_train)
    print('Best params: {}'.format(knn_search.best_params_))
    # predict
    Y_pred = knn_search.predict(X_val)
    mse = mean_squared_error(Y_val, Y_pred)
    print('MSE: {}'.format(mse))
    r2 = knn_search.score(X_val, Y_val)
    print('R^2: {}'.format(r2))
    res.loc[c, 'KNN'] = r2
    print()

For cluster 0:
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:   22.9s finished


Best params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 19, 'knn__weights': 'distance'}
MSE: 37.39444612918919
R^2: 0.5815981259249603

For cluster 1:
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.8s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:   22.4s finished


Best params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 23, 'knn__weights': 'distance'}
MSE: 45.162115993807326
R^2: 0.41031556176420503

For cluster 2:
Fitting 3 folds for each of 24 candidates, totalling 72 fits


...

Best params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 23, 'knn__weights': 'distance'}
MSE: 20.22060723138457
R^2: 0.5587874097668584

For cluster 28:
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:   22.6s finished


Best params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 29, 'knn__weights': 'distance'}
MSE: 18.699575264618257
R^2: 0.2895502672169412

For cluster 29:
Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done  72 out of  72 | elapsed:   26.8s finished


Best params: {'knn__metric': 'manhattan', 'knn__n_neighbors': 29, 'knn__weights': 'uniform'}
MSE: 2.7299488541038883
R^2: -0.010639605540229136



In [24]:
# save csv
res.to_csv('./pred_res.csv', index=False)

## Decision Tree

In [25]:
from sklearn.tree import DecisionTreeRegressor

In [26]:
# set pipeline
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('tree', DecisionTreeRegressor())])

In [27]:
# set grid search
params = {'tree__max_depth': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
tree_search = GridSearchCV(pipeline, params, cv=3, verbose=2, n_jobs=-1)

In [28]:
res['Decision Tree'] = 0
for c in dfs:
    print('For cluster {}:'.format(c))
    # train test split
    df_train, df_val = train_test_split(dfs[c], test_size=0.2, random_state=1207)
    # get X Y
    X_train = df_train.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_train = df_train['Checkin per Hour'] - df_train['Checkout per Hour']
    X_val = df_val.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_val = df_val['Checkin per Hour'] - df_val['Checkout per Hour']
    # shuffle
    X_train, Y_train = shuffle(X_train, Y_train, random_state=1207)
    # train model
    tree_search.fit(X_train, Y_train)
    print('Best params: {}'.format(tree_search.best_params_))
    # predict
    Y_pred = tree_search.predict(X_val)
    mse = mean_squared_error(Y_val, Y_pred)
    print('MSE: {}'.format(mse))
    r2 = tree_search.score(X_val, Y_val)
    print('R^2: {}'.format(r2))
    res.loc[c, 'Decision Tree'] = r2
    print()

For cluster 0:
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'tree__max_depth': 6}
MSE: 42.243773594812325
R^2: 0.5273395953236335

For cluster 1:
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'tree__max_depth': 6}
MSE: 47.2724816821951
R^2: 0.3827603912846004

For cluster 2:
Fitting 3 folds for each of 10 candidates, totalling 30 fits


...

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'tree__max_depth': 7}
MSE: 24.448229017132856
R^2: 0.4665409239283448

For cluster 28:
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Best params: {'tree__max_depth': 5}
MSE: 20.94292323534716
R^2: 0.20431913529067802

For cluster 29:
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Best params: {'tree__max_depth': 3}
MSE: 2.730812440894102
R^2: -0.010959309336767253



[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    0.6s finished


In [29]:
# save csv
res.to_csv('./pred_res.csv', index=False)

## Random Forest 

In [30]:
from sklearn.ensemble import RandomForestRegressor

In [31]:
# set pipeline
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('rf', RandomForestRegressor())])

In [32]:
# set grid search
params = {'rf__n_estimators': [10, 50, 100, 500],
          'rf__max_features': ['auto', 'log2', 'sqrt']}
rf_search = GridSearchCV(pipeline, params, cv=3, verbose=2, n_jobs=-1)

In [33]:
res['Random Forest'] = 0
for c in dfs:
    print('For cluster {}:'.format(c))
    # train test split
    df_train, df_val = train_test_split(dfs[c], test_size=0.2, random_state=1207)
    # get X Y
    X_train = df_train.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_train = df_train['Checkin per Hour'] - df_train['Checkout per Hour']
    X_val = df_val.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_val = df_val['Checkin per Hour'] - df_val['Checkout per Hour']
    # shuffle
    X_train, Y_train = shuffle(X_train, Y_train, random_state=1207)
    # train model
    rf_search.fit(X_train, Y_train)
    print('Best params: {}'.format(rf_search.best_params_))
    # predict
    Y_pred = rf_search.predict(X_val)
    mse = mean_squared_error(Y_val, Y_pred)
    print('MSE: {}'.format(mse))
    r2 = rf_search.score(X_val, Y_val)
    print('R^2: {}'.format(r2))
    res.loc[c, 'Random Forest'] = r2
    print()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


For cluster 0:
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   41.6s finished


Best params: {'rf__max_features': 'sqrt', 'rf__n_estimators': 500}
MSE: 38.970470063926946
R^2: 0.5639641873020087

For cluster 1:
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   43.6s finished


...

Best params: {'rf__max_features': 'sqrt', 'rf__n_estimators': 500}
MSE: 21.554706283105023
R^2: 0.5296774383648346

For cluster 28:
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   42.6s finished


Best params: {'rf__max_features': 'log2', 'rf__n_estimators': 500}
MSE: 19.138498495433787
R^2: 0.27287433273006567

For cluster 29:
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   47.0s finished


Best params: {'rf__max_features': 'log2', 'rf__n_estimators': 500}
MSE: 2.7892946187214616
R^2: -0.032609680200575086



In [34]:
# save csv
res.to_csv('./pred_res.csv', index=False)

## Xgboost 

In [35]:
import xgboost as xgb

In [36]:
# set pipeline
pipeline = Pipeline([('scaler', StandardScaler()),
                     ('xgb', xgb.XGBRegressor())])

In [37]:
# set grid search
params = {'xgb__min_child_weight': [1, 5, 10],
          'xgb__gamma': [0.5, 1, 1.5],
          'xgb__subsample': [0.6, 0.8, 1.0],
          'xgb__colsample_bytree': [0.6, 0.8, 1.0],
          'xgb__max_depth': [3, 4, 5]}
xgb_search = GridSearchCV(pipeline, params, cv=3, verbose=2, n_jobs=-1)

In [38]:
res['Xgboost'] = 0
for c in dfs:
    print('For cluster {}:'.format(c))
    # train test split
    df_train, df_val = train_test_split(dfs[c], test_size=0.2, random_state=1207)
    # get X Y
    X_train = df_train.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_train = df_train['Checkin per Hour'] - df_train['Checkout per Hour']
    X_val = df_val.drop(columns=['Checkin per Hour', 'Checkout per Hour'])
    Y_val = df_val['Checkin per Hour'] - df_val['Checkout per Hour']
    # shuffle
    X_train, Y_train = shuffle(X_train, Y_train, random_state=1207)
    # train model
    xgb_search.fit(X_train, Y_train)
    print('Best params: {}'.format(xgb_search.best_params_))
    # predict
    Y_pred = xgb_search.predict(X_val)
    mse = mean_squared_error(Y_val, Y_pred)
    print('MSE: {}'.format(mse))
    r2 = xgb_search.score(X_val, Y_val)
    print('R^2: {}'.format(r2))
    res.loc[c, 'Xgboost'] = r2
    print()

For cluster 0:
Fitting 3 folds for each of 243 candidates, totalling 729 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   35.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 729 out of 729 | elapsed:  4.1min finished


[12:50:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best params: {'xgb__colsample_bytree': 0.8, 'xgb__gamma': 1, 'xgb__max_depth': 4, 'xgb__min_child_weight': 1, 'xgb__subsample': 1.0}
MSE: 37.339908019494736
R^2: 0.5822083461492642

For cluster 1:
Fitting 3 folds for each of 243 candidates, totalling 729 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   36.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 729 out of 729 | elapsed:  3.8min finished


[12:54:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best params: {'xgb__colsample_bytree': 1.0, 'xgb__gamma': 1.5, 'xgb__max_depth': 4, 'xgb__min_child_weight': 1, 'xgb__subsample': 1.0}
MSE: 45.79301523298527
R^2: 0.4020778728240091

For cluster 2:
Fitting 3 folds for each of 243 candidates, totalling 729 fits


...

[14:28:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best params: {'xgb__colsample_bytree': 0.8, 'xgb__gamma': 1, 'xgb__max_depth': 4, 'xgb__min_child_weight': 1, 'xgb__subsample': 1.0}
MSE: 21.42245664717054
R^2: 0.5325631184910712

For cluster 28:
Fitting 3 folds for each of 243 candidates, totalling 729 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   57.8s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 729 out of 729 | elapsed:  4.6min finished


[14:32:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best params: {'xgb__colsample_bytree': 0.8, 'xgb__gamma': 0.5, 'xgb__max_depth': 4, 'xgb__min_child_weight': 1, 'xgb__subsample': 1.0}
MSE: 19.445878211359254
R^2: 0.26119610828101225

For cluster 29:
Fitting 3 folds for each of 243 candidates, totalling 729 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   41.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 729 out of 729 | elapsed:  3.9min finished


[14:36:40] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best params: {'xgb__colsample_bytree': 0.6, 'xgb__gamma': 0.5, 'xgb__max_depth': 3, 'xgb__min_child_weight': 1, 'xgb__subsample': 1.0}
MSE: 2.7412253484794986
R^2: -0.014814215555545518



In [39]:
# save csv
res.to_csv('./pred_res.csv', index=False)